In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

# 定义Neo-Hookean模型的应力计算函数
# Define the stress calculation function of the Neo-Hookean model
def neo_hookean_stress(F, mu, lamb):
    Ic = np.trace(np.dot(F.T, F))
    J = np.linalg.det(F)
    stress = mu/J*(F - np.eye(3)) + lamb/J*np.log(J)*np.eye(3)
    return stress

# 生成样本数据（形变张量和应力张量）
# generate sample data (deformation tensor and stress tensor)
n_samples = 1000
mu_true = 1.0
lamb_true = 10.0
noise_std = 0.05
F_samples = np.random.uniform(0.8, 1.2, size=(n_samples, 3, 3))
stress_samples = np.zeros((n_samples, 3, 3))
for i in range(n_samples):
    stress_samples[i] = neo_hookean_stress(F_samples[i], mu_true, lamb_true)
    noise = np.random.normal(scale=noise_std, size=(3, 3))
    stress_samples[i] += noise

# 定义似然函数（概率模型）
# Define the likelihood function (probability model)
def likelihood(params, F, stress):
    mu, lamb, noise_std = params
    n_samples = F.shape[0]
    ll = 0
    for i in range(n_samples):
        model_stress = neo_hookean_stress(F[i], mu, lamb)
        ll += norm.logpdf(stress[i], loc=model_stress, scale=noise_std).sum()
    return -ll

# 最大化似然函数，得到最优的模型参数估计
# Maximize the likelihood function to get the optimal model parameter estimation
res = minimize(likelihood, x0=[1.0, 10.0, 0.1], args=(F_samples, stress_samples), method='L-BFGS-B')
mu_est, lamb_est, noise_std_est = res.x
print('Estimated mu:', mu_est)
print('Estimated lamb:', lamb_est)
print('Estimated noise_std:', noise_std_est)


/var/folders/y5/_38246qd6zd0lmzngc4zlt8h0000gn/T/ipykernel_16892/3529092758.py:9: RuntimeWarning: invalid value encountered in log
  stress = mu/J*(F - np.eye(3)) + lamb/J*np.log(J)*np.eye(3)


Estimated mu: 1.0
Estimated lamb: 10.0
Estimated noise_std: 0.1
